# 🌱 Классификация сортов растений по изображениям семян

Этот notebook демонстрирует полный процесс обучения модели для классификации семян различных растений.

## Содержание
1. Загрузка и исследование данных
2. Предобработка и аугментация
3. Создание и обучение модели
4. Оценка результатов
5. Визуализация

## 1. Импорт библиотек

In [ ]:
import sys
import os

# Добавляем путь к src
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import tensorflow as tf
from tensorflow import keras

# Импорт наших модулей
from data_preprocessing import SeedDataPreprocessor
from model import SeedClassificationModel
from utils import plot_training_history, plot_confusion_matrix, plot_sample_predictions

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Установка стиля графиков
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Загрузка и исследование данных

In [ ]:
# Параметры
DATA_DIR = '../data/raw/seeds'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Создание препроцессора
preprocessor = SeedDataPreprocessor(
    data_dir=DATA_DIR,
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Загрузка путей к изображениям
image_paths, labels = preprocessor.load_data_paths()

print(f"\nВсего изображений: {len(image_paths)}")
print(f"Количество классов: {len(preprocessor.class_names)}")
print(f"Классы: {preprocessor.class_names}")

### Создание DataFrame и анализ распределения

In [ ]:
# Создание DataFrame
df = preprocessor.create_dataframe(image_paths, labels)

# Отображение первых строк
print("\nПервые 5 записей:")
display(df.head())

# Статистика
print("\nСтатистика по классам:")
display(df['class'].value_counts())

### Визуализация распределения классов

In [ ]:
plt.figure(figsize=(14, 6))

class_counts = df['class'].value_counts().sort_values(ascending=False)
colors = plt.cm.viridis(np.linspace(0, 1, len(class_counts)))

bars = plt.bar(range(len(class_counts)), class_counts.values, color=colors)
plt.xlabel('Класс', fontsize=14)
plt.ylabel('Количество изображений', fontsize=14)
plt.title('Распределение классов в датасете', fontsize=16, fontweight='bold')
plt.xticks(range(len(class_counts)), class_counts.index, rotation=45, ha='right')

# Добавляем значения
for bar, count in zip(bars, class_counts.values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(count)}', ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

### Просмотр примеров изображений

In [ ]:
from PIL import Image
import random

# Выбираем по одному примеру из каждого класса
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for idx, class_name in enumerate(preprocessor.class_names[:10]):
    # Берём случайное изображение из класса
    class_images = df[df['class'] == class_name]['filepath'].values
    img_path = random.choice(class_images)
    
    # Загрузка изображения
    img = Image.open(img_path)
    
    # Отображение
    axes[idx].imshow(img)
    axes[idx].set_title(class_name, fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.suptitle('Примеры изображений из каждого класса', fontsize=16, y=1.02)
plt.show()

## 3. Разделение данных и создание генераторов

In [ ]:
# Разделение на train/val/test
train_df, val_df, test_df = preprocessor.split_data(df)

print("\nРазмеры выборок:")
print(f"Train: {len(train_df)}")
print(f"Validation: {len(val_df)}")
print(f"Test: {len(test_df)}")

In [ ]:
# Создание генераторов с аугментацией
train_gen, val_gen, test_gen = preprocessor.create_data_generators(
    train_df, val_df, test_df, augmentation=True
)

print(f"\nКоличество классов: {len(train_gen.class_indices)}")
print(f"Маппинг классов: {train_gen.class_indices}")

### Визуализация аугментированных данных

In [ ]:
# Берём один батч для визуализации аугментации
aug_images, aug_labels = next(train_gen)

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.flatten()

for i in range(8):
    axes[i].imshow(aug_images[i])
    class_idx = np.argmax(aug_labels[i])
    class_name = list(train_gen.class_indices.keys())[class_idx]
    axes[i].set_title(class_name, fontsize=11)
    axes[i].axis('off')

plt.tight_layout()
plt.suptitle('Примеры аугментированных изображений', fontsize=14, y=1.02)
plt.show()

## 4. Создание модели

In [ ]:
# Параметры модели
NUM_CLASSES = len(train_gen.class_indices)
MODEL_TYPE = 'resnet50'  # Можно изменить на 'efficientnet' или 'vgg16'
LEARNING_RATE = 0.0001

# Создание модели
model_builder = SeedClassificationModel(
    num_classes=NUM_CLASSES,
    img_size=IMG_SIZE,
    model_type=MODEL_TYPE
)

model = model_builder.build_model(trainable_base=False)
model_builder.compile_model(learning_rate=LEARNING_RATE)

print(f"\n✅ Модель {MODEL_TYPE} создана и скомпилирована")
print(f"Параметров в модели: {model.count_params():,}")

### Архитектура модели

In [ ]:
# Краткое описание архитектуры
model.summary()

## 5. Обучение модели

In [ ]:
# Параметры обучения
EPOCHS = 50

# Получаем callbacks
callbacks = model_builder.get_callbacks(checkpoint_path='../models/best_model.h5')

# Обучение
print("🚀 Начало обучения...")
history = model_builder.train(
    train_generator=train_gen,
    val_generator=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

print("\n✅ Обучение завершено!")

### Визуализация процесса обучения

In [ ]:
# График обучения
plot_training_history(history, save_path='../reports/training_history.png')

# Отображение в ноутбуке
from IPython.display import Image as IPImage
display(IPImage('../reports/training_history.png'))

## 6. Оценка модели на тестовой выборке

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Загружаем лучшую модель
best_model = keras.models.load_model('../models/best_model.h5')

# Предсказания на тестовой выборке
test_gen.reset()
predictions = best_model.predict(test_gen, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = test_gen.classes

# Имена классов
class_names = list(train_gen.class_indices.keys())

# Метрики
test_loss, test_accuracy = best_model.evaluate(test_gen, verbose=0)

print("\n" + "="*70)
print("📊 РЕЗУЛЬТАТЫ НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*70)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print("\n" + "="*70)

### Classification Report

In [ ]:
# Детальный отчёт
report = classification_report(y_true, y_pred, target_names=class_names)
print("\nClassification Report:")
print(report)

### Confusion Matrix

In [ ]:
# Матрица ошибок
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, class_names, save_path='../reports/confusion_matrix.png')

# Отображение
display(IPImage('../reports/confusion_matrix.png'))

### Примеры предсказаний

In [ ]:
# Визуализация примеров
plot_sample_predictions(
    best_model,
    test_gen,
    class_names,
    num_samples=16,
    save_path='../reports/predictions_sample.png'
)

# Отображение
display(IPImage('../reports/predictions_sample.png'))

## 7. Анализ ошибок

In [ ]:
# Находим наиболее частые ошибки
errors = []
for true_idx, pred_idx in zip(y_true, y_pred):
    if true_idx != pred_idx:
        errors.append((class_names[true_idx], class_names[pred_idx]))

# Подсчёт частоты ошибок
from collections import Counter
error_counts = Counter(errors)
most_common_errors = error_counts.most_common(10)

print("\nТоп-10 наиболее частых ошибок:")
print("-" * 70)
for (true_class, pred_class), count in most_common_errors:
    print(f"{true_class:<20} → {pred_class:<20} : {count} раз")
print("-" * 70)

## 8. Сохранение финальной модели

In [ ]:
# Сохранение модели
model_builder.save_model('../models/final_model.h5')

# Сохранение конфигурации
import json

config = {
    'model_type': MODEL_TYPE,
    'num_classes': NUM_CLASSES,
    'img_size': IMG_SIZE[0],
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'learning_rate': LEARNING_RATE,
    'test_accuracy': float(test_accuracy),
    'class_names': class_names
}

with open('../models/config.json', 'w') as f:
    json.dump(config, f, indent=4)

print("✅ Модель и конфигурация сохранены")

## 9. Выводы

В этом проекте мы:

1. ✅ Загрузили и исследовали датасет семян
2. ✅ Провели предобработку и аугментацию данных
3. ✅ Создали модель на базе ResNet50 с Transfer Learning
4. ✅ Обучили модель с использованием callbacks
5. ✅ Оценили качество на тестовой выборке
6. ✅ Проанализировали результаты и ошибки

### Итоговые метрики:
- **Test Accuracy:** указать после выполнения
- **F1-Score (macro):** указать после выполнения

### Возможные улучшения:
- Fine-tuning базовой модели
- Использование ансамблей моделей
- Дополнительная аугментация
- Балансировка классов